In [1]:
import rebound
import numpy as np
import os

In [121]:
# We use a global variable to store the value of the eccentricity
global planet_e_s, mutual_inc
planet_e_s, mutual_inc = [], []
def heartbeat(sim_pointer):
    # The function argument is a pointer to the simulation:
    # Here we get its contents:
    sim = sim_pointer.contents
    # e = sim.particles[2].e
    planet_e_s.append(sim.particles[1].e)
    
    orbits = sim.orbits()
    mutual_inc.append(orbits[1].inc-orbits[0].inc)

In [3]:
sim.stop_server()
del sim

In [2]:
sim = rebound.Simulation()
# sim.start_server(port=1234)
sim.widget(size=(400,400))

sim.dt = np.pi*1e-2
sim.integrator = "ias15"
sim.add(m=1.)
sim.add(m=1.e-3, a=10)
sim.add(primary=sim.particles[0], m=1e-5, a=1, e=0.01, inc=75/180*np.pi)
sim.move_to_com()

REBOUND Webserver listening on http://localhost:1234 ...


In [4]:
sim.integrate(1e8)

KeyboardInterrupt: 

In [122]:
sim = rebound.Simulation()
# Start the visualization web server.
# Point your browser to http://localhost:1234
sim.start_server(port=1234)

# Setup constants
sim.dt = np.pi*1e-2
sim.integrator = "ias15"
sim.heartbeat = heartbeat

# Initial conditions
# Central object
sim.add(m=1.)
star = sim.particles[0]
# test particle (inner planet)
sim.add(m=1e-3, a=1. ,e=0.)
planet = sim.particles[1]
# outer perturber
inc_p = 85
m_p = 1
a_p = 10.
vy_p = np.cos(inc_p/180.*np.pi)*np.sqrt((star.m+m_p)/a_p)
vz_p = np.sin(inc_p/180.*np.pi)*np.sqrt((star.m+m_p)/a_p)
sim.add(m=m_p, x=a_p, vy=vy_p, vz=vz_p) # perturber

In [123]:
sim.move_to_com()

In [124]:
sim.integrate(1e5)
print(sim.steps_done)
print(sim.walltime)

758020
9.871681999965348


In [125]:
planet_e_s = np.array(planet_e_s)
# print(planet_e_s)
np.savetxt('planet_e.txt', planet_e_s)

In [ ]:
mutual_inc = np.array(mutual_inc)
# print(planet_e_s)
np.savetxt('mutual_inc.txt', mutual_inc)

In [ ]:
# num = sim.steps_done//10
# print(num)
# sim.save_to_file(f"kozai_{num}.bin", step=10)

In [ ]:
sim = None